In [1]:
# Base dependancies:
import pandas as pd
import numpy as np
from collections import defaultdict

################################################
# Make an empty DefaultDict
# fill dictionary {year: dataframe}
# build hierarchal dataframe using dictionary
################################################

# Example of data location, uses year 2010
# https://raw.githubusercontent.com/SethDKelly/NiceRideMN/master/Nice_Ride_data/2010/NiceRide_station_2010.csv

station_dict = defaultdict()
ridership_dict = defaultdict()

for year in [2010 + x for x in range(8)] :
    station_dict[year] = pd.read_csv("https://raw.githubusercontent.com/SethDKelly/NiceRideMN/master/Nice_Ride_data/" \
                             +str(year)+"/NiceRide_station_"+str(year)+".csv")

    ridership_dict[year] = pd.read_csv("https://raw.githubusercontent.com/SethDKelly/NiceRideMN/master/Nice_Ride_data/" \
                             +str(year)+"/NiceRide_trip_history_"+str(year)+".csv")

NR_station = pd.concat(station_dict)
NR_ridership = pd.concat(ridership_dict)

In [2]:
# For KMeans model building
from sklearn.cluster import KMeans

# For Feature Optimizations
from sklearn.decomposition import PCA

#########################################################################
# This sections builds the clusters for each (2010-2017) years data
# The code uses KMeans clustering with a K value of 6
# Appends to the DF `NR_station` the cluster each station belongs to
# into a new column `Cluster`
########################################################################

NR_station['Cluster'] = int(0)

for year in [2010 + x for x in range(8)] :
    
    distance_matrix = pd.DataFrame() # Reinstantiate the  Distance Matrix DataFrame for clean run
    
    # Read in the distance matrix for the particular year
    distance_matrix = pd.read_csv("https://raw.githubusercontent.com/SethDKelly/NiceRideMN/master/Nice_Ride_data/" \
                               +str(year)+"/distance_matrix_"+str(year)+".csv",index_col=0)
    
    # Optimize distance matrix to two primary x-y components
    pca = PCA(n_components=2).fit_transform(distance_matrix)
    
    #Append to the DF `NR_station` the cluster each station belongs to in column `Cluster`
    NR_station.loc[year, 'Cluster'] =  KMeans(n_clusters=6, n_init=200).fit(pca).labels_

In [3]:
# returns a dataframe with the total count of rides from station start and station end
ride_counts = NR_ridership.loc[2017,:].drop(['Start_date', 'Start_name', 'End_date', 'End_name', 'duration', 'account'], axis=1)
ride_counts['counts'] = 0
ride_counts = ride_counts.groupby(by=['Start_id', 'End_id'],axis=0, as_index=False)['counts'].count()

In [18]:
import networkx as nx

#####################
# Generate graph data
#####################


graph = nx.from_pandas_edgelist(ride_counts,\
                                source = 'Start_id', \
                                target = 'End_id', \
                                edge_attr = 'counts', \
                                create_using = nx.Graph())

### Below should be calculated and saved, calculation on the fly is time intensive

In [16]:
def get_edge_coor(graph, _station) :

################################################################
# Takes two arguements, graph and _station
# graph must be of type nx.Graph()
# _station must have columns: 'Termina', 'Latitude', 'Longitude'
#
# function takes the edges in graph
# finds the nodes locations:
# node1(x, y) / node1(long/lat), node2(x, y) / node2(long/lat)
# creates 3 different arrays: xs, ys, alpha
# xs are the x values for node1/2, ys are the y values for node1/2
# alpha is the `weight` of the node, to be used as edge alpha value
# returns xs, ys, alpha
################################################################

    assert (type(graph) == nx.Graph),"arg graph must be of type Graph not of type DiGraph, MultiGraph, or MultiDiGraph"
    assert 'Terminal' in _station.columns, "Column named Terminal must be in arg DataFrame"
    assert 'Longitude' in _station.columns, "Column named Longitude must be in arg DataFrame"
    assert 'Latitude' in _station.columns, "Column named Latitude must be in arg DataFrame"

    xs = []
    ys = []
    alpha = []
    edge = dict(xs=[], ys=[], alpha=[])
    # example: { ..., ('30001.0', '300005.0', {'counts': 243}), ... }
    # u is origin_node, v is terminus_node, d is data (weights)
    
    for u, v, d in graph.edges.data('counts', default = 0):
        
        edge['xs'].append([_station[_station.Terminal == u].Longitude.values[0], \
                  _station[_station.Terminal == v].Longitude.values[0]])

        edge['ys'].append([_station[_station.Terminal == u].Latitude.values[0], \
                  _station[_station.Terminal == v].Latitude.values[0]])
        edge['alpha'].append(d) # rescale for later alpha normalization
        
    # create values of alpha that are between 0 and 1
    edge['alpha'] = [a / max(edge['alpha']) for a in edge['alpha']]
    
    return edge

In [19]:
# Perform necessary imports for Bokeh plotting
from bokeh.io import output_file, show, curdoc
from bokeh.plotting import figure

# Import the models modules
from bokeh.models import ColumnDataSource, HoverTool, Slider, ColorMapper

# Import the layout modules
from bokeh.layouts import widgetbox, column

# To build graph as an application
from bokeh.io import curdoc

# For cluster coloring
from bokeh.palettes import Spectral6

# Make the ColumnDataSource: source
node_source = ColumnDataSource(data={
    'x'        : NR_station.loc[2017,:].Longitude,
    'y'        : NR_station.loc[2017,:].Latitude,
    'terminal' : NR_station.loc[2017,:].Terminal,
    'station'  : NR_station.loc[2017,:].Station,
    'cluster'  : NR_station.loc[2017,:].Cluster
    })

# Setting boundaries for x-y axis ranges
xmin, xmax = (NR_station.loc[:,:].Longitude.values.min() - .01) , (NR_station.loc[:,:].Longitude.values.max() + .01)
ymin, ymax = (NR_station.loc[:,:].Latitude.values.min() - .01) , (NR_station.loc[:,:].Latitude.values.max() + .01)

# Creating hovertool tip
hover = HoverTool(tooltips=[('Terminal', '@terminal'), 
                           ('Name', '@station'),
                           ('Location', '($x, $y)')])

# Create the figure: plot
plot = figure(plot_height=600, plot_width=900,
              x_range = (xmin, xmax), y_range = (ymin, ymax),
              tools=[hover,'tap', 'box_zoom', 'reset', 'wheel_zoom', 'pan', 'lasso_select'])

# Set the x-axis label
plot.xaxis.axis_label = 'Longitude'

# Set the y-axis label
plot.yaxis.axis_label = 'Latitude'

# Add the color mapper to the circle glyph
r_circles = plot.circle(x='x', y='y', source=node_source,
            fill_alpha=0.8, legend='cluster') # Add color mapper by cluster, add node size by 

# Set the legend.location attribute of the plot to 'top_right'
plot.legend.location = 'top_right'

###############################
# Section to add NetworkX edges
###############################

edge_source = ColumnDataSource(data = get_edge_coor(graph, NR_station.loc[2017,:]))
r_lines = plot.multi_line(xs = 'xs',ys = 'ys', source=edge_source,
                          line_width=1.5, alpha='alpha', color='navy')

######################################################
# Section is to set up year slider widget
# Allow Users to update graph by year using the slider
######################################################
'''
# Define the callback function: update_plot
def update_plot(attr, old, new):

    # set the `yr` name to `slider.value` and `source.data = new_data`
    yr = slider.value
    
    new_node_data = {
        'x'        : NR_station.loc[yr, :].Longitude,
        'y'        : NR_station.loc[yr, :].Latitude,
        'terminal' : NR_station.loc[yr, :].Terminal,
        'station'  : NR_station.loc[yr, :].Station,
        'cluster'  : NR_station.loc[yr, :].Cluster,
    }
    
    new_edge_data = ColumnDataSource(data = get_edge_coor(graph, NR_station.loc[yr,:]))
    
    # Assign updated data to: *_source.data
    node_source.data = new_node_data
    edge_source.data = new_edge_data
    
    # Add title to figure: plot.title.text
    plot.title.text = 'NiceRideMN data for %d' % yr


# Make a slider object: slider
slider = Slider(start=2010, end=2017, step=1, value=2017, title='Year')

# Attach the callback to the 'value' property of slider
slider.on_change('value', update_plot)
'''
######################################################
# Section is to set up cluster buttons
# Allow Users to choose which clusters to show
######################################################

######################################################
# Section is to setup how the plot and widgets are displayed
######################################################
'''
# Make a row layout of widgetbox(slider) and plot and add it to the current document
layout = column(plot, widgetbox(slider))
curdoc().add_root(layout)


# Add the plot to the current document and add a title
curdoc().add_root(plot)
curdoc().title = 'Nice Ride stations'
'''


output_file('TEST')
show(plot)